In [1]:
using Distributed
# add processors if we don't have enough
# if length(workers()) < 6
# 	addprocs(6)
# end

@everywhere begin
	using Distributions
	using Plots
	using Turing
	using StatsPlots
end

In [2]:
@everywhere begin
	n_schools = 8
	y = [28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0] # estimated treatment effects
	σ = [15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0] # standard error of the effect estimate
end

In [3]:
@everywhere begin
	
	@model function schools(y, σ)
		μ ~ Normal(0, 5)
		τ ~ truncated(Cauchy(0, 5), lower = 0)
		θ ~ filldist(Normal(μ, τ), n_schools)
		for i in 1:n_schools
			y[i] ~ Normal(θ[i], σ[i])
		end
	end

	@model function school_reparam(y, σ)
		μ ~ Normal(0, 5)
		τ ~ truncated(Cauchy(0, 5), lower = 0)
		θ ~ filldist(Normal(0, 1), n_schools)
		# y .~ ForEach(μ .+ τ .* θ, σ)
		for i in 1:n_schools
			y[i] ~ Normal(μ + τ * θ[i], σ[i])
		end
	end
end


#### Inference with the No U-Turn Sampler (NUTS)

In [ ]:
c1 = sample(schools(y, σ), NUTS(0.8),MCMCDistributed(), 2_000,12,n_adapts=1000)

In [ ]:
c2 = sample(school_reparam(y, σ), NUTS(0.8),MCMCDistributed(), 2_000,12,n_adapts=1000)

#### Emcee

In [ ]:
c3 = sample(schools(y, σ), Emcee(50),20_000)

In [ ]:
c4 = sample(school_reparam(y, σ), Emcee(50),20_000)

## Using MicroCanonical HMC

In [11]:
@everywhere begin
	using MicroCanonicalHMC
	n_adapts = 2_000 # adaptation steps
	tev = 0.001 # target energy variance
	mchmc = MCHMC(n_adapts, tev; adaptive = true)
end

In [ ]:
c5 = sample(schools(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

In [ ]:
c6 = sample(school_reparam(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

In [ ]:
plot(c1)

In [ ]:
plot(c2)

In [ ]:
plot(c3)

In [ ]:
plot(c4)

In [ ]:
plot(c5)

In [ ]:
plot(c6)